# Import libraries

In [143]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# Load and clean data

In [144]:
# Read in the data
toronto = pd.read_csv("toronto.csv")

In [145]:
# Delete the empty boroughs
assigned = (toronto["Borough"] != "Not assigned")
toronto = toronto[assigned]

In [146]:
# Reindex data
toronto.index = range(toronto.shape[0])
toronto.head(10)

Postal Code      Borough                                    Neighbourhood
0         M1B  Scarborough                                   Malvern, Rouge
1         M1C  Scarborough           Rouge Hill, Port Union, Highland Creek
2         M1E  Scarborough                Guildwood, Morningside, West Hill
3         M1G  Scarborough                                           Woburn
4         M1H  Scarborough                                        Cedarbrae
5         M1J  Scarborough                              Scarborough Village
6         M1K  Scarborough      Kennedy Park, Ionview, East Birchmount Park
7         M1L  Scarborough                  Golden Mile, Clairlea, Oakridge
8         M1M  Scarborough  Cliffside, Cliffcrest, Scarborough Village West
9         M1N  Scarborough                      Birch Cliff, Cliffside West

In [147]:
toronto.shape

(103, 3)

In [148]:
# Read in the locations
locations = pd.read_csv("coordinates.csv")
locations.sort_values(by=["Postal Code"], inplace=True)

In [149]:
# Add coordinates to main dataframe
toronto["Latitude"] = locations["Latitude"]
toronto["Longitude"] = locations["Longitude"]
toronto.head(10)

Postal Code      Borough                                    Neighbourhood  \
0         M1B  Scarborough                                   Malvern, Rouge   
1         M1C  Scarborough           Rouge Hill, Port Union, Highland Creek   
2         M1E  Scarborough                Guildwood, Morningside, West Hill   
3         M1G  Scarborough                                           Woburn   
4         M1H  Scarborough                                        Cedarbrae   
5         M1J  Scarborough                              Scarborough Village   
6         M1K  Scarborough      Kennedy Park, Ionview, East Birchmount Park   
7         M1L  Scarborough                  Golden Mile, Clairlea, Oakridge   
8         M1M  Scarborough  Cliffside, Cliffcrest, Scarborough Village West   
9         M1N  Scarborough                      Birch Cliff, Cliffside West   

    Latitude  Longitude  
0  43.806686 -79.194353  
1  43.784535 -79.160497  
2  43.763573 -79.188711  
3  43.770992 -79.216917  
4  43.773136 -79.239476  
5  43.744734 -79.239476  
6  43.727929 -79.262029  
7  43.711112 -79.284577  
8  43.716316 -79.239476  
9  43.692657 -79.264848

# Visualize locations

In [150]:
latitude = 43.7280
longitude = -79.3888
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.728, -79.3888.


In [151]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**Please use https://nbviewer.jupyter.org/github/shekharmore/Coursera_Capstone/blob/master/week3/week3_submission_3.ipynb  to render the map properly**

# Get data using Foursquare

**Define FourSquare credentials**

In [152]:
CLIENT_ID = 'MNC40RH332TUVZZQUMDQQNDLSTJ03CKDA4J0RZ5QXDJM2XOL' # your Foursquare ID
CLIENT_SECRET = 'UN4DRN0DWN23YHTE2BJ25QQV1H1YHX30ZKGX2H01UT4OECBG' # your Foursquare Secret
VERSION = '20200101' # Foursquare API version

print('Credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentials:
CLIENT_ID: MNC40RH332TUVZZQUMDQQNDLSTJ03CKDA4J0RZ5QXDJM2XOL
CLIENT_SECRET:UN4DRN0DWN23YHTE2BJ25QQV1H1YHX30ZKGX2H01UT4OECBG


**Define function to get venues in all neighbourhoods in Toronto**

In [153]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [154]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

**Get nearby venues for all neighbourhoods**

In [157]:
toronto_venues = getNearbyVenues(names=toronto['Neighbourhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

In [158]:
toronto_venues.shape

(2136, 7)

In [159]:
toronto_venues.head()

Neighbourhood  Neighbourhood Latitude  \
0                          Malvern, Rouge               43.806686   
1  Rouge Hill, Port Union, Highland Creek               43.784535   
2  Rouge Hill, Port Union, Highland Creek               43.784535   
3       Guildwood, Morningside, West Hill               43.763573   
4       Guildwood, Morningside, West Hill               43.763573   

   Neighbourhood Longitude                           Venue  Venue Latitude  \
0               -79.194353                         Wendy’s       43.807448   
1               -79.160497           Royal Canadian Legion       43.782533   
2               -79.160497  Scarborough Historical Society       43.788755   
3               -79.188711                  RBC Royal Bank       43.766790   
4               -79.188711               G & G Electronics       43.765309   

   Venue Longitude        Venue Category  
0       -79.199056  Fast Food Restaurant  
1       -79.163085                   Bar  
2       -79.162438        History Museum  
3       -79.191151                  Bank  
4       -79.191537     Electronics Store

**Check how many venues were returned for each neighborhood**

In [160]:
toronto_venues.groupby('Neighbourhood').count()

Neighbourhood Latitude  \
Neighbourhood                                                                
Agincourt                                                                4   
Alderwood, Long Branch                                                  11   
Bathurst Manor, Wilson Heights, Downsview North                         21   
Bayview Village                                                          4   
Bedford Park, Lawrence Manor East                                       23   
Berczy Park                                                             58   
Birch Cliff, Cliffside West                                              4   
Brockton, Parkdale Village, Exhibition Place                            23   
Business reply mail Processing Centre, South Ce...                      17   
CN Tower, King and Spadina, Railway Lands, Harb...                      14   
Caledonia-Fairbanks                                                      4   
Canada Post Gateway Processing Centre                                   12   
Cedarbrae                                                                8   
Central Bay Street                                                      62   
Christie                                                                17   
Church and Wellesley                                                    77   
Clarks Corners, Tam O'Shanter, Sullivan                                 13   
Cliffside, Cliffcrest, Scarborough Village West                          3   
Commerce Court, Victoria Hotel                                         100   
Davisville                                                              33   
Davisville North                                                         7   
Del Ray, Mount Dennis, Keelsdale and Silverthorn                         4   
Don Mills                                                               27   
Dorset Park, Wexford Heights, Scarborough Town ...                       6   
Downsview                                                               18   
Dufferin, Dovercourt Village                                            16   
East Toronto, Broadview North (Old East York)                            3   
Eringate, Bloordale Gardens, Old Burnhamthorpe,...                       7   
Fairview, Henry Farm, Oriole                                            64   
First Canadian Place, Underground city                                 100   
Forest Hill North & West, Forest Hill Road Park                          4   
Garden District, Ryerson                                               100   
Glencairn                                                                4   
Golden Mile, Clairlea, Oakridge                                         10   
Guildwood, Morningside, West Hill                                        8   
Harbourfront East, Union Station, Toronto Islands                      100   
High Park, The Junction South                                           24   
Hillcrest Village                                                        5   
Humber Summit                                                            1   
Humberlea, Emery                                                         2   
Humewood-Cedarvale                                                       4   
India Bazaar, The Beaches West                                          22   
Kennedy Park, Ionview, East Birchmount Park                              7   
Kensington Market, Chinatown, Grange Park                               58   
Kingsview Village, St. Phillips, Martin Grove G...                       4   
Lawrence Manor, Lawrence Heights                                        13   
Lawrence Park                                                            3   
Leaside                                                                 32   
Little Portugal, Trinity                                                44   
Malvern, Rouge                                                           1

In [161]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 271 uniques categories.


# Analyze each neighbourhood

In [162]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Neighbourhood  Accessories Store  \
0                          Malvern, Rouge                  0   
1  Rouge Hill, Port Union, Highland Creek                  0   
2  Rouge Hill, Port Union, Highland Creek                  0   
3       Guildwood, Morningside, West Hill                  0   
4       Guildwood, Morningside, West Hill                  0   

   Afghan Restaurant  Airport  Airport Food Court  Airport Lounge  \
0                  0        0                   0               0   
1                  0        0                   0               0   
2                  0        0                   0               0   
3                  0        0                   0               0   
4                  0        0                   0               0   

   Airport Service  Airport Terminal  American Restaurant  Antique Shop  \
0                0                 0                    0             0   
1                0                 0                    0             0   
2                0                 0                    0             0   
3                0                 0                    0             0   
4                0                 0                    0             0   

   Aquarium  Art Gallery  Art Museum  Arts & Crafts Store  Asian Restaurant  \
0         0            0           0                    0                 0   
1         0            0           0                    0                 0   
2         0            0           0                    0                 0   
3         0            0           0                    0                 0   
4         0            0           0                    0                 0   

   Athletics & Sports  Auto Garage  Auto Workshop  BBQ Joint  Baby Store  \
0                   0            0              0          0           0   
1                   0            0              0          0           0   
2                   0            0              0          0           0   
3                   0            0              0          0           0   
4                   0            0              0          0           0   

   Bagel Shop  Bakery  Bank  Bar  Baseball Field  Baseball Stadium  \
0           0       0     0    0               0                 0   
1           0       0     0    1               0                 0   
2           0       0     0    0               0                 0   
3           0       0     1    0               0                 0   
4           0       0     0    0               0                 0   

   Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  Beer Store  \
0                   0      0                0         0           0   
1                   0      0                0         0           0   
2                   0      0                0         0           0   
3                   0      0                0         0           0   
4                   0      0                0         0           0   

   Belgian Restaurant  Bike Shop  Bistro  Board Shop  Boat or Ferry  \
0                   0          0       0           0              0   
1                   0          0       0           0              0   
2                   0          0       0           0              0   
3                   0          0       0           0              0   
4                   0          0       0           0              0   

   Bookstore  Boutique  Brazilian Restaurant  Breakfast Spot  Brewery  \
0          0         0                     0               0        0   
1          0         0                     0               0        0   
2          0         0                     0               0        0   
3          0         0                     0               0        0   
4          0         0                     0               0        0   

   Bridal Shop  Bubble Tea Shop  Building  Burger Joint  Burrito Place  \
0            0                0         0             0  

In [163]:
toronto_onehot.shape

(2136, 272)

In [164]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

Neighbourhood  Accessories Store  \
0                                        Agincourt                0.0   
1                           Alderwood, Long Branch                0.0   
2  Bathurst Manor, Wilson Heights, Downsview North                0.0   
3                                  Bayview Village                0.0   
4                Bedford Park, Lawrence Manor East                0.0   

   Afghan Restaurant  Airport  Airport Food Court  Airport Lounge  \
0                0.0      0.0                 0.0             0.0   
1                0.0      0.0                 0.0             0.0   
2                0.0      0.0                 0.0             0.0   
3                0.0      0.0                 0.0             0.0   
4                0.0      0.0                 0.0             0.0   

   Airport Service  Airport Terminal  American Restaurant  Antique Shop  \
0              0.0               0.0             0.000000           0.0   
1              0.0               0.0             0.000000           0.0   
2              0.0               0.0             0.000000           0.0   
3              0.0               0.0             0.000000           0.0   
4              0.0               0.0             0.043478           0.0   

   Aquarium  Art Gallery  Art Museum  Arts & Crafts Store  Asian Restaurant  \
0       0.0          0.0         0.0                  0.0               0.0   
1       0.0          0.0         0.0                  0.0               0.0   
2       0.0          0.0         0.0                  0.0               0.0   
3       0.0          0.0         0.0                  0.0               0.0   
4       0.0          0.0         0.0                  0.0               0.0   

   Athletics & Sports  Auto Garage  Auto Workshop  BBQ Joint  Baby Store  \
0            0.000000          0.0            0.0        0.0         0.0   
1            0.090909          0.0            0.0        0.0         0.0   
2            0.000000          0.0            0.0        0.0         0.0   
3            0.000000          0.0            0.0        0.0         0.0   
4            0.000000          0.0            0.0        0.0         0.0   

   Bagel Shop  Bakery      Bank  Bar  Baseball Field  Baseball Stadium  \
0         0.0     0.0  0.000000  0.0             0.0               0.0   
1         0.0     0.0  0.000000  0.0             0.0               0.0   
2         0.0     0.0  0.095238  0.0             0.0               0.0   
3         0.0     0.0  0.250000  0.0             0.0               0.0   
4         0.0     0.0  0.000000  0.0             0.0               0.0   

   Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  Beer Store  \
0                 0.0    0.0              0.0       0.0         0.0   
1                 0.0    0.0              0.0       0.0         0.0   
2                 0.0    0.0              0.0       0.0         0.0   
3                 0.0    0.0              0.0       0.0         0.0   
4                 0.0    0.0              0.0       0.0         0.0   

   Belgian Restaurant  Bike Shop  Bistro  Board Shop  Boat or Ferry  \
0                 0.0        0.0     0.0         0.0            0.0   
1                 0.0        0.0     0.0         0.0            0.0   
2                 0.0        0.0     0.0         0.0            0.0   
3                 0.0        0.0     0.0         0.0            0.0   
4                 0.0        0.0     0.0         0.0            0.0   

   Bookstore  Boutique  Brazilian Restaurant  Breakfast Spot  Brewery  \
0        0.0       0.0                   0.0            0.25      0.0   
1        0.0       0.0                   0.0            0.00      0.0   
2        0.0       0.0                   0.0            0.00      0.0   
3        0.0       0.0                   0.0            0.00      0.0   
4        0.0       0.0                   0.0            0.00      0.0   

   Bridal Shop  Bubble Tea Shop  Building  Burger Joint  Burri

In [165]:
toronto_grouped.shape

(97, 272)

**Print top 5 most common venues in each neighbourhood**

In [166]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1                     Lounge  0.25
2  Latin American Restaurant  0.25
3               Skating Rink  0.25
4          Accessories Store  0.00


----Alderwood, Long Branch----
                venue  freq
0         Pizza Place  0.18
1            Pharmacy  0.09
2  Athletics & Sports  0.09
3      Sandwich Place  0.09
4                 Gym  0.09


----Bathurst Manor, Wilson Heights, Downsview North----
           venue  freq
0           Bank  0.10
1    Coffee Shop  0.10
2  Deli / Bodega  0.05
3    Bridal Shop  0.05
4  Shopping Mall  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2  Japanese Restaurant  0.25
3   Chinese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.09
1    Sushi Restaurant  0.09
2          Restaurant  0.09
3         Coffee Sh

4  Miscellaneous Shop  0.00


----Leaside----
                    venue  freq
0             Coffee Shop  0.09
1  Furniture / Home Store  0.06
2            Burger Joint  0.06
3                    Bank  0.06
4     Sporting Goods Shop  0.06


----Little Portugal, Trinity----
                           venue  freq
0                            Bar  0.11
1                     Restaurant  0.07
2               Asian Restaurant  0.07
3                    Men's Store  0.05
4  Vegetarian / Vegan Restaurant  0.05


----Malvern, Rouge----
                       venue  freq
0       Fast Food Restaurant   1.0
1  Middle Eastern Restaurant   0.0
2                      Motel   0.0
3        Moroccan Restaurant   0.0
4        Monument / Landmark   0.0


----Milliken, Agincourt North, Steeles East, L'Amoreaux East----
                       venue  freq
0                 Playground   0.5
1                       Park   0.5
2          Accessories Store   0.0
3  Middle Eastern Restaurant   0.0
4               

          venue  freq
0      Pharmacy  0.18
1   Curling Ice  0.09
2           Spa  0.09
3  Skating Rink  0.09
4    Beer Store  0.09


----York Mills West----
                       venue  freq
0                       Park   0.5
1          Convenience Store   0.5
2          Accessories Store   0.0
3  Middle Eastern Restaurant   0.0
4                      Motel   0.0


----York Mills, Silver Hills----
                      venue  freq
0         Martial Arts Dojo   0.5
1                 Cafeteria   0.5
2            Massage Studio   0.0
3            Medical Center   0.0
4  Mediterranean Restaurant   0.0




**Function to sort the venues**

In [167]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

**Create new dataframe and display top 10 venues for each neighbourhood**

In [168]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighbourhood 1st Most Common Venue  \
0                                        Agincourt                Lounge   
1                           Alderwood, Long Branch           Pizza Place   
2  Bathurst Manor, Wilson Heights, Downsview North                  Bank   
3                                  Bayview Village                  Café   
4                Bedford Park, Lawrence Manor East           Coffee Shop   

       2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0  Latin American Restaurant        Breakfast Spot          Skating Rink   
1               Dance Studio                   Gym           Coffee Shop   
2                Coffee Shop         Shopping Mall              Pharmacy   
3                       Bank    Chinese Restaurant   Japanese Restaurant   
4                 Restaurant      Sushi Restaurant    Italian Restaurant   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0      Doner Restaurant    Dim Sum Restaurant                 Diner   
1          Skating Rink              Pharmacy                   Pub   
2           Supermarket      Sushi Restaurant            Restaurant   
3      Department Store    Dim Sum Restaurant                 Diner   
4        Sandwich Place              Pharmacy                   Pub   

  8th Most Common Venue 9th Most Common Venue     10th Most Common Venue  
0        Discount Store   Distribution Center                    Dog Run  
1        Sandwich Place    Athletics & Sports                       Pool  
2         Deli / Bodega   Fried Chicken Joint  Middle Eastern Restaurant  
3        Discount Store   Distribution Center                    Dog Run  
4          Liquor Store               Butcher            Thai Restaurant

# Cluster the neighbourhoods

**Run *k*-means to cluster the neighborhood into 5 clusters**

In [169]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [170]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

Postal Code      Borough                           Neighbourhood   Latitude  \
0         M1B  Scarborough                          Malvern, Rouge  43.806686   
1         M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2         M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3         M1G  Scarborough                                  Woburn  43.770992   
4         M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.194353             2.0  Fast Food Restaurant         Deli / Bodega   
1 -79.160497             1.0                   Bar        History Museum   
2 -79.188711             1.0    Mexican Restaurant     Electronics Store   
3 -79.216917             1.0           Coffee Shop     Korean Restaurant   
4 -79.239476             1.0      Hakka Restaurant           Gas Station   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0           Event Space  Ethiopian Restaurant     Electronics Store   
1           Yoga Studio      Doner Restaurant    Dim Sum Restaurant   
2        Breakfast Spot                  Bank   Rental Car Location   
3           Yoga Studio      Doner Restaurant    Dim Sum Restaurant   
4                  Bank   Fried Chicken Joint    Athletics & Sports   

         6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0  Eastern European Restaurant             Drugstore            Donut Shop   
1                        Diner        Discount Store   Distribution Center   
2               Medical Center         Moving Target          Intersection   
3                        Diner        Discount Store   Distribution Center   
4              Thai Restaurant  Caribbean Restaurant                Bakery   

  9th Most Common Venue 10th Most Common Venue  
0      Doner Restaurant                Dog Run  
1               Dog Run             Donut Shop  
2           Yoga Studio                  Diner  
3               Dog Run              Drugstore  
4      Doner Restaurant                Dog Run

# Visualize the clusters

In [171]:
toronto_merged.dropna(inplace=True)

In [172]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Please use https://nbviewer.jupyter.org/github/shekharmore/Coursera_Capstone/blob/master/week3/week3_submission_3.ipynb to render the map properly**

# Examine the clusters

**Cluster 1**

In [173]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

Neighbourhood  Cluster Labels  \
14  Milliken, Agincourt North, Steeles East, L'Amo...             0.0   
23                                    York Mills West             0.0   
25                                          Parkwoods             0.0   
40      East Toronto, Broadview North (Old East York)             0.0   
44                                      Lawrence Park             0.0   
50                                           Rosedale             0.0   
74                                Caledonia-Fairbanks             0.0   
79           North Park, Maple Leaf Park, Upwood Park             0.0   
90      The Kingsway, Montgomery Road, Old Mill North             0.0   
98                                             Weston             0.0   

   1st Most Common Venue 2nd Most Common Venue       3rd Most Common Venue  \
14                  Park            Playground                 Yoga Studio   
23                  Park     Convenience Store                 Yoga Studio   
25                  Park     Food & Drink Shop                 Yoga Studio   
40                  Park           Coffee Shop           Convenience Store   
44                  Park           Swim School                    Bus Line   
50                  Park                 Trail                  Playground   
74                  Park                  Pool               Women's Store   
79                  Park                Bakery  Construction & Landscaping   
90                  Park                 River                 Yoga Studio   
98                  Park     Convenience Store                 Yoga Studio   

   4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
14               Dog Run      Department Store          Dessert Shop   
23      Doner Restaurant          Dessert Shop    Dim Sum Restaurant   
25               Dog Run          Dessert Shop    Dim Sum Restaurant   
40           Yoga Studio      Doner Restaurant    Dim Sum Restaurant   
44           Yoga Studio   Distribution Center          Dessert Shop   
50      Department Store          Dessert Shop    Dim Sum Restaurant   
74               Dog Run      Department Store          Dessert Shop   
79           Yoga Studio      Doner Restaurant    Dim Sum Restaurant   
90   Distribution Center      Department Store          Dessert Shop   
98      Doner Restaurant          Dessert Shop    Dim Sum Restaurant   

   7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
14    Dim Sum Restaurant                 Diner        Discount Store   
23                 Diner        Discount Store   Distribution Center   
25                 Diner        Discount Store   Distribution Center   
40                 Diner        Discount Store   Distribution Center   
44    Dim Sum Restaurant                 Diner        Discount Store   
50                 Diner        Discount Store   Distribution Center   
74    Dim Sum Restaurant                 Diner        Discount Store   
79                 Diner        Discount Store   Distribution Center   
90    Dim Sum Restaurant                 Diner        Discount Store   
98                 Diner        Discount Store   Distribution Center   

   10th Most Common Venue  
14    Distribution Center  
23                Dog Run  
25       Doner Restaurant  
40                Dog Run  
44                Dog Run  
50                Dog Run  
74    Distribution Center  
79                Dog Run  
90                Dog Run  
98                Dog Run

**Cluster 2**

In [174]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

Neighbourhood  Cluster Labels  \
1               Rouge Hill, Port Union, Highland Creek             1.0   
2                    Guildwood, Morningside, West Hill             1.0   
3                                               Woburn             1.0   
4                                            Cedarbrae             1.0   
6          Kennedy Park, Ionview, East Birchmount Park             1.0   
7                      Golden Mile, Clairlea, Oakridge             1.0   
8      Cliffside, Cliffcrest, Scarborough Village West             1.0   
9                          Birch Cliff, Cliffside West             1.0   
10   Dorset Park, Wexford Heights, Scarborough Town...             1.0   
11                                   Wexford, Maryvale             1.0   
12                                           Agincourt             1.0   
13             Clarks Corners, Tam O'Shanter, Sullivan             1.0   
15                       Steeles West, L'Amoreaux West             1.0   
17                                   Hillcrest Village             1.0   
18                        Fairview, Henry Farm, Oriole             1.0   
19                                     Bayview Village             1.0   
20                            York Mills, Silver Hills             1.0   
21                             Willowdale, Newtonbrook             1.0   
22                         Willowdale, Willowdale East             1.0   
24                         Willowdale, Willowdale West             1.0   
26                                           Don Mills             1.0   
27                                           Don Mills             1.0   
28     Bathurst Manor, Wilson Heights, Downsview North             1.0   
29                     Northwood Park, York University             1.0   
30                                           Downsview             1.0   
31                                           Downsview             1.0   
32                                           Downsview             1.0   
33                                           Downsview             1.0   
34                                    Victoria Village             1.0   
35                     Parkview Hill, Woodbine Gardens             1.0   
36                                    Woodbine Heights             1.0   
37                                         The Beaches             1.0   
38                                             Leaside             1.0   
39                                    Thorncliffe Park             1.0   
41                        The Danforth West, Riverdale             1.0   
42                      India Bazaar, The Beaches West             1.0   
43                                     Studio District             1.0   
45                                    Davisville North             1.0   
46                   North Toronto West, Lawrence Park             1.0   
47                                          Davisville             1.0   
48                         Moore Park, Summerhill East             1.0   
49   Summerhill West, Rathnelly, South Hill, Forest...             1.0   
51                         St. James Town, Cabbagetown             1.0   
52                                Church and Wellesley             1.0   
53                           Regent Park, Harbourfront             1.0   
54                            Garden District, Ryerson             1.0   
55                                      St. James Town             1.0   
56                                         Berczy Park             1.0   
57                                  Central Bay Street             1.0   
58                            Richmond, Adelaide, King             1.0   
59   Harbourfront East, Union Station, Toronto Islands             1.0   
60            Toronto Dominion Centre, Design Exchange             1.0   
61                      Commerce Court, Victoria Hotel             1.0   
62                   Bedford Park, Lawrence M

**Cluster 3**

In [175]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

Neighbourhood  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0  Malvern, Rouge             2.0  Fast Food Restaurant         Deli / Bodega   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0           Event Space  Ethiopian Restaurant     Electronics Store   

         6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0  Eastern European Restaurant             Drugstore            Donut Shop   

  9th Most Common Venue 10th Most Common Venue  
0      Doner Restaurant                Dog Run

**Cluster 4**

In [176]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

Neighbourhood  Cluster Labels 1st Most Common Venue  \
5  Scarborough Village             3.0            Playground   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
5     Convenience Store           Yoga Studio      Doner Restaurant   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
5          Dessert Shop    Dim Sum Restaurant                 Diner   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
5        Discount Store   Distribution Center                Dog Run

**Cluster 5**

In [177]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

Neighbourhood  Cluster Labels  \
91  Old Mill South, King's Mill Park, Sunnylea, Hu...             4.0   
97                                   Humberlea, Emery             4.0   

            1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
91                 Baseball Field           Yoga Studio               Dog Run   
97  Paper / Office Supplies Store        Baseball Field           Yoga Studio   

   4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
91          Dessert Shop    Dim Sum Restaurant                 Diner   
97      Doner Restaurant    Dim Sum Restaurant                 Diner   

   7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
91        Discount Store   Distribution Center      Doner Restaurant   
97        Discount Store   Distribution Center               Dog Run   

   10th Most Common Venue  
91         Farmers Market  
97             Donut Shop

# THE END